<a href="https://colab.research.google.com/github/Pedro-Boechat/trainee/blob/main/periodo3v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.decomposition import PCA

# Binário

In [268]:
df = pd.read_csv('/content/winequality.csv')

In [269]:
df.tail()

,Unnamed: 0,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,wine_is_red
6492,6492,7.7,0.32,0.62,10.6,0.036,56.0,153.0,0.99780,3.13,0.44,8.9,6,0
6493,6493,6.2,0.22,0.28,2.2,0.040,24.0,125.0,0.99170,3.19,0.48,10.5,6,0
6494,6494,7.9,0.33,0.28,31.6,0.053,35.0,176.0,1.01030,3.15,0.38,8.8,6,0
6495,6495,7.0,0.30,0.28,2.2,0.042,21.0,177.0,0.99166,3.20,0.57,11.4,5,0
6496,6496,6.4,0.22,0.49,7.5,0.054,42.0,151.0,0.99480,3.27,0.52,10.1,6,0


In [270]:
df.isna().sum()

Unnamed: 0              0
fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
wine_is_red             0
dtype: int64

In [271]:
df = df.drop(columns='Unnamed: 0')

In [272]:
colunas = df.columns

## Normalizar

In [273]:
for i in colunas:
  df[i] = (df[i] - df[i].min()) / (df[i].max() - df[i].min())

In [274]:
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,wine_is_red
0,0.173554,0.353333,0.048193,0.023006,0.102990,0.052083,0.041475,0.128205,0.651163,0.308989,0.449275,0.5,1.0
1,0.363636,0.613333,0.054217,0.026074,0.093023,0.020833,0.071429,0.187777,0.465116,0.185393,0.144928,0.5,1.0
2,0.214876,0.106667,0.295181,0.079755,0.073090,0.083333,0.262673,0.136688,0.224806,0.426966,0.362319,0.5,0.0
3,0.231405,0.100000,0.156627,0.010736,0.059801,0.052083,0.281106,0.121265,0.496124,0.213483,0.289855,0.5,0.0
4,0.280992,0.106667,0.174699,0.036810,0.044850,0.055556,0.255760,0.134953,0.496124,0.258427,0.304348,0.5,0.0


## Verificar balanceamento

In [275]:
df['wine_is_red'].value_counts()
# Levemente desbalanceado. Verificar

0.0    4898
1.0    1599
Name: wine_is_red, dtype: int64

In [276]:
from sklearn.utils import resample
# Separar a maioria e minoria
df_majority = df[df['wine_is_red']==0]
df_minority = df[df['wine_is_red']==1]
# Downsample
df_majority_downsampled = resample(df_majority, 
                                 replace=False,    
                                 n_samples=1599)
# Combinar a maioria downsampled com a minoria
df = pd.concat([df_majority_downsampled, df_minority])

## Implementando KNN

In [277]:
from sklearn.model_selection import train_test_split

X = df[colunas.drop('wine_is_red')]
Y = df['wine_is_red']
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.2)

## PCA

Testei com e sem PCA, e não mudou muita coisa

In [278]:
#pca = PCA(.95)

In [279]:
#pca.fit(X)

In [280]:
#pca.n_components_

In [281]:
#X = pca.transform(X)

## Train

In [282]:
print(X_test)
print(X_train)
print(y_test)
X_test = np.array(X_test)
X_train = np.array(X_train)
y_train = np.array(y_train)
y_test = np.array(y_test)

      fixed acidity  volatile acidity  ...   alcohol   quality
5611       0.512397          0.233333  ...  0.289855  0.333333
318        0.297521          0.260000  ...  0.362319  0.333333
1848       0.305785          0.133333  ...  0.623188  0.833333
4357       0.429752          0.493333  ...  0.260870  0.333333
4093       0.214876          0.373333  ...  0.260870  0.333333
...             ...               ...  ...       ...       ...
80         0.165289          0.140000  ...  0.492754  0.500000
5199       0.388430          0.260000  ...  0.449275  0.500000
2841       0.239669          0.120000  ...  0.347826  0.500000
3858       0.214876          0.173333  ...  0.144928  0.166667
5366       0.206612          0.086667  ...  0.130435  0.333333

[640 rows x 12 columns]
      fixed acidity  volatile acidity  ...   alcohol   quality
6111       0.280992          0.386667  ...  0.246377  0.333333
4242       0.272727          0.393333  ...  0.202899  0.333333
5478       0.247934          0

In [283]:
distancias = [np.sqrt(np.sum((x_test - X_train)**2, axis=1)) for x_test in X_test]
len(distancias)

640

In [284]:
from scipy import stats

pred = np.array([])
k = 7
for i in range(len(distancias)):
  idxkneigh = distancias[i].argsort()[:k] # Pega os indices das k menores distancias
  vizinhos = [y_train[idxkneigh[j]] for j in range(k)] # Busca qual a classificação desses indices: red ou não
  mode_info = stats.mode(vizinhos) # Pega a moda das classificações dos KNN
  pred = np.append(pred, mode_info[0]) # Prediz

In [285]:
accuracy = np.in1d(pred,y_test) # Comparar onde os resultados são corretos
precisao = np.count_nonzero(pred == True)/len(distancias)

In [288]:
precisao

0.5125